In [1]:
from dataset import BPI2012Dataset
from parameters.dataset import BPI2012Parameters
from utils.save import get_json_dict
from parameters.training import TrainingParameters, LossParameters, OptimizerParameters
from parameters.model import PredNextBERTParameters
from model import PredNextBERT
from controller import TrainingController

In [2]:
bpi_2012_parms = BPI2012Parameters()

In [3]:
get_json_dict(bpi_2012_parms)

{'file_path': './data/event_logs/BPI_Challenge_2012.xes',
 'preprocessed_folder_path': './data/preprocessed/BPI_Challenge_2012_with_resource',
 'include_types': ['A', 'O', 'W'],
 'include_complete_only': True}

In [4]:
dataset = BPI2012Dataset(bpi_2012_parms)


| Preprocessed data loaded successfully: ./data/preprocessed/BPI_Challenge_2012_with_resource/AOW_CompleteOnly 


In [5]:
model_params = PredNextBERTParameters()

In [6]:
get_json_dict(model_params)

{'num_layers': 4,
 'model_dim': 32,
 'feed_forward_dim': 64,
 'num_heads': 4,
 'dropout_rate': 0.1}

In [7]:
model = PredNextBERT(
    activity_vocab = dataset.activity_vocab,
    parameters = model_params
)

In [8]:
train_params = TrainingParameters(
    stop_epoch=50,
)

In [9]:
get_json_dict(train_params)

{'stop_epoch': 50,
 'batch_size': 128,
 'verbose_freq': 250,
 'run_validation_freq': 80,
 'train_test_split_portion': [0.8, 0.1],
 'random_seed': 12345}

In [10]:
from parameters.enum import SelectableLoss, SelectableLrScheduler, SelectableOptimizer

In [11]:
loss_params = LossParameters(
    loss=SelectableLoss.CrossEntropy,
    from_logits=True
)

In [12]:
get_json_dict(loss_params)

{'loss': 'CrossEntropy', 'from_logits': True}

In [13]:
optim_params = OptimizerParameters(
    optimizer=SelectableOptimizer.Adam,
    lr_scheduler=SelectableLrScheduler.BERTScheduler,
)

In [14]:
get_json_dict(optim_params)

{'optimizer': 'Adam',
 'learning_rate': 0.005,
 'l2': 1e-10,
 'lr_scheduler': 'BERTScheduler',
 'lr_step_scheduler_step': 800,
 'lr_setp_scheduler_gamma': 0.8,
 'lr_exp_decay_scheduler_step': 1000,
 'lr_exp_decay_scheduler_rate': 0.96,
 'lr_exp_decay_scheduler_staircase': True,
 'SGD_momentum': 0.9}

In [15]:
controller = TrainingController(
    dataset,
    model,
    train_params,
    optim_params,
    loss_params,
)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  


In [16]:
controller.model_info()

Model: "PredNextBERT"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
predict_next_encoder (Predic multiple                  35008     
_________________________________________________________________
dense_24 (Dense)             multiple                  858       
Total params: 35,866
Trainable params: 35,866
Non-trainable params: 0
_________________________________________________________________


In [17]:
# tensorboard --logdir logs/gradient_tape --host localhost --port 8088
test_accuracy = controller.train()


| Training records in logs/gradient_tape/20210704-061448 

| Total epochs: 50 

| Total steps: 4100 

| Start epoch 0 

| Evaluation result | Loss [3.0928] | Accuracy [0.1105]  

| Start epoch 1 

| Evaluation result | Loss [2.6773] | Accuracy [0.3009]  

| Start epoch 2 

| Evaluation result | Loss [1.8532] | Accuracy [0.4875]  

| Start epoch 3 

| Evaluation result | Loss [1.2777] | Accuracy [0.6446]  

| Start epoch 4 

| Evaluation result | Loss [1.0530] | Accuracy [0.6698]  

| Start epoch 5 

| Evaluation result | Loss [0.9200] | Accuracy [0.7062]  

| Start epoch 6 

| Evaluation result | Loss [0.8062] | Accuracy [0.7270]  

| Start epoch 7 

| Evaluation result | Loss [0.7215] | Accuracy [0.7469]  

| Start epoch 8 

| Evaluation result | Loss [0.6827] | Accuracy [0.7548]  

| Start epoch 9 

| Evaluation result | Loss [0.6561] | Accuracy [0.7618]  

| Start epoch 10 

| Evaluation result | Loss [0.6366] | Accuracy [0.7680]  

| Start epoch 11 

| Evaluation result | Loss [0.

In [18]:
save_folder_path = model.get_folder_path("bpi_2012_bert_training.ipynb", test_accuracy , dataset.info_str())

In [19]:
controller.save_parameters(save_folder_path)
model.save(save_folder_path)


| Parameters saved successfully to: ./SavedModels/0.7876_PredNextBERT_AOW_CompleteOnly_2021-07-04 06:28:09.605710  

| Vocabs saved successfully to: ./SavedModels/0.7876_PredNextBERT_AOW_CompleteOnly_2021-07-04 06:28:09.605710  

| Model saved successfully to: ./SavedModels/0.7876_PredNextBERT_AOW_CompleteOnly_2021-07-04 06:28:09.605710  


In [20]:
# controller.load_parameters(save_folder_path)
# model.load(save_folder_path)